In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
from skg import nsphere_fit
import scipy.io as sio
import json
from tqdm import tqdm
import os

%matplotlib qt

In [2]:
# get 5 random frames from the videoes
if not os.path.exists('random_frames'):
    os.makedirs('random_frames')

videos = list(filter(lambda x: x.endswith('.avi'), os.listdir()))
n_frames_to_get = 2
for video in videos:
    print(video)
    cap = cv2.VideoCapture(video)
    n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if 'phase_1' in video:
        n_frames_phase1 = n_frames
    elif 'phase_2' in video:
        n_frames_phase2 = n_frames
    for i in range(n_frames_to_get):
        frame_number = np.random.randint(n_frames)
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
        ret, frame = cap.read()
        # save the frame
        cv2.imwrite(f'random_frames/{video}_{frame_number}.png', frame)
    cap.release()

20hrs-wingless-orcoctrl-yy_phase_2.avi
20hrs-wingless-orcoctrl-yy_phase_1.avi


In [3]:
# make a tracking video
import skvideo.io
from tqdm import tqdm
# load the tracking data
# phase 1
phase1 = []
for i in range(1, 8):
    df = pd.read_csv('20hrs-wingless-orcoctrl-yy_phase_1/20hrs-wingless-orcoctrl-yy_phase_1-trackfeat.csv/fly{}.csv'.format(i))
    df = df[['pos x', 'pos y','ori', 'major axis len', 'minor axis len']]
    phase1.append(df)
# phase 2
phase2 = []
for i in range(1, 8):
    df = pd.read_csv('20hrs-wingless-orcoctrl-yy_phase_2/20hrs-wingless-orcoctrl-yy_phase_2-trackfeat.csv/fly{}.csv'.format(i))
    df = df[['pos x', 'pos y','ori', 'major axis len', 'minor axis len']]
    phase2.append(df)

# make sure the number of frames is the same
assert np.all([len(df) == len(phase1[0]) for df in phase1])
assert np.all([len(df) == len(phase2[0]) for df in phase2])
assert n_frames_phase1 == len(phase1[0])
assert n_frames_phase2 == len(phase2[0])

In [4]:
# define 7 colors in RGB
colors = np.array([[0, 0, 1], [0, 1, 0], [1, 0, 0], [1, 0, 1], [0, 1, 1], [1, 1, 0], [1, 1, 1]])
colors = np.int32(colors*255)

In [5]:
# phase 1
cap = cv2.VideoCapture('20hrs-wingless-orcoctrl-yy_phase_1.avi')
frame_rate = cap.get(cv2.CAP_PROP_FPS)
writer = skvideo.io.FFmpegWriter("20hrs-wingless-orcoctrl-yy_phase_1_tracking.mp4", inputdict={'-r': str(frame_rate)})
for i in tqdm(range(n_frames_phase1)):
    ret, frame = cap.read()
    # make a copy of the frame
    frame_copy = frame.copy()
    for j in range(7):
        # get the position and orientation of the fly
        # if any of the position is nan, skip the fly
        if np.any(np.isnan(phase1[j].iloc[i])):
            continue
        x = int(phase1[j].iloc[i]['pos x'])
        y = int(phase1[j].iloc[i]['pos y'])
        ori = -phase1[j].iloc[i]['ori']
        major_axis_len = phase1[j].iloc[i]['major axis len']
        minor_axis_len = phase1[j].iloc[i]['minor axis len']
        # draw the fly
        cv2.ellipse(frame_copy, center=(x, y), axes=(int(major_axis_len/2), int(minor_axis_len/2)), angle=ori*180/np.pi, startAngle=0, endAngle=360, color=colors[j].tolist(), thickness=-1)
    # combine the frame with the tracking data with transparency
    frame = cv2.addWeighted(frame, 0.7, frame_copy, 0.3, 0)
    writer.writeFrame(frame)
writer.close()
cap.release()

# phase 2
cap = cv2.VideoCapture('20hrs-wingless-orcoctrl-yy_phase_2.avi')
frame_rate = cap.get(cv2.CAP_PROP_FPS)
writer = skvideo.io.FFmpegWriter("20hrs-wingless-orcoctrl-yy_phase_2_tracking.mp4", inputdict={'-r': str(frame_rate)})
for i in tqdm(range(n_frames_phase2)):
    ret, frame = cap.read()
    # make a copy of the frame
    frame_copy = frame.copy()
    for j in range(7):
        # get the position and orientation of the fly
        # if any of the position is nan, skip the fly
        if np.any(np.isnan(phase2[j].iloc[i])):
            continue
        x = int(phase2[j].iloc[i]['pos x'])
        y = int(phase2[j].iloc[i]['pos y'])
        ori = -phase2[j].iloc[i]['ori']
        major_axis_len = phase2[j].iloc[i]['major axis len']
        minor_axis_len = phase2[j].iloc[i]['minor axis len']
        # draw the fly
        cv2.ellipse(frame_copy, center=(x, y), axes=(int(major_axis_len/2), int(minor_axis_len/2)), angle=ori*180/np.pi, startAngle=0, endAngle=360, color=colors[j].tolist(), thickness=-1)
    # combine the frame with the tracking data with transparency
    frame = cv2.addWeighted(frame, 0.7, frame_copy, 0.3, 0)
    writer.writeFrame(frame)
writer.close()
cap.release()

100%|██████████| 115201/115201 [44:56<00:00, 42.73it/s]   
